#### *4) Hiperparametre Optimizasyonu*

Hiperparametre optimizasyonu, bir modelin performansını artırmak amacıyla, modelin belirli parametrelerinin (hiperparametreler) en iyi kombinasyonunu bulma sürecidir. GridSearchCV ve RandomizedSearchCV, bu amaçla yaygın olarak kullanılan iki yöntemdir.

**Hiperparametre Optimizasyon Süreci:**

***Hiperparametrelerin İncelenmesi:***

Her makine öğrenmesi modelinin performansını etkileyen bir dizi hiperparametre vardır. 

Örneğin:

SVM: C, kernel, gamma gibi hiperparametreler.
Random Forest: n_estimators, max_depth, min_samples_split, min_samples_leaf gibi hiperparametreler.
XGBoost: n_estimators, learning_rate, max_depth, subsample, colsample_bytree gibi hiperparametreler.

***GridSearchCV veya RandomizedSearchCV ile Optimizasyon:***

GridSearchCV: Tüm hiperparametre kombinasyonlarını denemek için kullanılır. Daha kapsamlıdır, ancak işlem süresi uzun olabilir.
RandomizedSearchCV: Belirli bir sayıda rastgele hiperparametre kombinasyonu dener. Daha hızlıdır, ancak optimum sonucu kaçırma olasılığı vardır.

(BELKİ SİLİNEBİLİR)
(XGBOOST ve random forest İÇİN OPTİMİZASYON YAPILDI
AMA SEÇİLEN MODEL İÇİN YAPILSA DAHA İYİ OLUR!!!!!

belki diğer modeller için de yapılabilir!!!)

**Gerekli kütüphane içe aktarımı ve veri seti okuması :**

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Veri setini yükleme
df = pd.read_csv('breast-cancer.csv')

In [28]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

**Verilerin test/eğitim olarak ayrılması ve ölçeklendirilmesi :**

In [9]:
# Bağımlı ve bağımsız değişkenlerin seçimi
X = df.drop(columns=['id', 'diagnosis'])  # id sütunu çıkarıldı, diagnosis hedef değişken olarak belirlendi
y = df['diagnosis']  # hedef değişken

# Veri setini eğitim ve test olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Özellikleri ölçeklendirme
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

**Örnek optimizasyon : XGBoost**

*İlk model :*

In [13]:
#XGBoost

# 'diagnosis' sütununu sayısal verilere dönüştürme
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

# Hedef değişkenlerin yeniden tanımlanması
y = df['diagnosis']

# Veri setini yeniden ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model eğitimi ve tahmin adımlarını tekrarlama
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

# Tahmin ve değerlendirme
y_pred_xgb = xgb_model.predict(X_test)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("XGBoost Doğruluk Skoru:", accuracy_xgb)
print(classification_report(y_test, y_pred_xgb))

XGBoost Doğruluk Skoru: 0.956140350877193
              precision    recall  f1-score   support

           0       0.96      0.97      0.97        71
           1       0.95      0.93      0.94        43

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



*Optimize edilmiş model :*

In [35]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# Örnek: XGBoost Hiperparametre Optimizasyonu

# Hiperparametreler için bir grid oluşturma
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# RandomizedSearchCV ile optimizasyon
xgb_random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=xgb_param_grid, n_iter=50, cv=5, n_jobs=-1, verbose=2, random_state=42)
xgb_random_search.fit(X_train, y_train)

# En iyi hiperparametreler
print("En iyi XGBoost hiperparametreleri: ", xgb_random_search.best_params_)
print("En iyi XGBoost doğruluk skoru: ", xgb_random_search.best_score_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
En iyi XGBoost hiperparametreleri:  {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.2, 'colsample_bytree': 0.7}
En iyi XGBoost doğruluk skoru:  0.9802197802197803


*Açıklamalar:*

rf_param_grid ve xgb_param_grid: Modelin hiperparametrelerini belirlemek için kullanılan grid (kılavuz).

best_params_: En iyi performansı gösteren hiperparametreler.

best_score_: Hiperparametre optimizasyonu sonucu elde edilen en iyi doğruluk skoru.

*Optimum Modelin Belirlenmesi:*

Hiperparametre optimizasyonu tamamlandıktan sonra, en iyi hiperparametreler ve buna bağlı olarak elde edilen en iyi model performansı belirlenir. Bu aşamadan sonra, en iyi performansı gösteren model ile tahminler yapılabilir ve modelin genel performansı daha da değerlendirilebilir.

(SİLİNEBİLİR)

In [11]:
#Destek Vektör Makineleri (SVM)

svm_model = SVC()
svm_model.fit(X_train, y_train)

# Tahmin ve değerlendirme
y_pred_svm = svm_model.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("SVM Doğruluk Skoru:", accuracy_svm)
print(classification_report(y_test, y_pred_svm))

SVM Doğruluk Skoru: 0.9766081871345029
              precision    recall  f1-score   support

           B       0.98      0.98      0.98       108
           M       0.97      0.97      0.97        63

    accuracy                           0.98       171
   macro avg       0.97      0.97      0.97       171
weighted avg       0.98      0.98      0.98       171



In [20]:
from sklearn.model_selection import GridSearchCV

# Hiperparametreler için bir grid oluşturma
svm_param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

# GridSearchCV ile optimizasyon
svm_grid_search = GridSearchCV(estimator=svm_model, param_grid=svm_param_grid, cv=5, n_jobs=-1, verbose=2)
svm_grid_search.fit(X_train, y_train)

# En iyi hiperparametreler
print("En iyi SVM hiperparametreleri: ", svm_grid_search.best_params_)
print("En iyi SVM doğruluk skoru: ", svm_grid_search.best_score_)

# En iyi model ile tahmin yapma
y_pred_svm_optimized = svm_grid_search.best_estimator_.predict(X_test)
accuracy_svm_optimized = accuracy_score(y_test, y_pred_svm_optimized)
print("Optimize edilmiş SVM Doğruluk Skoru:", accuracy_svm_optimized)
print(classification_report(y_test, y_pred_svm_optimized))

Fitting 5 folds for each of 64 candidates, totalling 320 fits



KeyboardInterrupt

